![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace%20in%20Spark%20NLP%20-%20WhisperForCTC.ipynb)

## Import WhisperForCTC models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- This feature is only in `Spark NLP 5.1.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- The Whisper model was introduced in `Spark NLP 5.1.0 and requires Spark versions 3.4.0 and up.`
- Official models are supported, but not all custom models may work.

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.32.0`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q transformers==4.32.0 tensorflow==2.11.0

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use the [whisper-tiny](https://huggingface.co/openai/whisper-tiny) model from HuggingFace as an example
- In addition to `TFWhisperForCTCModel` we also need to save the `WhisperProcessor`. This is the same for every model, these are assets needed for preprocessing inside Spark NLP.

In [ ]:
MODEL_NAME = "openai/whisper-tiny"
EXPORT_PATH = f"exported_tf/{MODEL_NAME}"
assets_folder = f"{EXPORT_PATH}/assets"

Exporting this model involves several steps. We need to

1. separate the audio encoder and token decoder and their cache tensors
3. create a wrapper to create the right model signatures
4. export the preprocessor to the `assets` folder

Don't worry if this next step seems overwhelming. Once you run the next cell everything should be exported to the right place!

In [ ]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import (
    convert_variables_to_constants_v2,
)
from transformers import TFWhisperForConditionalGeneration, WhisperProcessor

encoder_signature = [
    {
        "input_features": tf.TensorSpec(
            (None, None, None), tf.float32, name="input_features"
        )
    }
]

decoder_init_signature = [
    {
        "decoder_input_ids_init": tf.TensorSpec(
            (None, None), tf.int32, name="decoder_input_ids_init"
        ),
        "encoder_state_init": tf.TensorSpec(
            (None, None, None), tf.float32, name="encoder_state_init"
        ),
    }
]

decoder_cache_signature = [
    {
        "decoder_input_ids": tf.TensorSpec(
            (None, None), tf.int32, name="decoder_input_ids"
        ),
        "encoder_state": tf.TensorSpec(
            (None, None, None), tf.float32, name="encoder_state"
        ),
        "decoder_past_key_values": tf.TensorSpec(
            (
                4,
                2,
                None,
                None,
                None,
                64,
            ),  # (layers, key-values, batch_size, num_heads, decoder_sequence_length, embed_size_per_head)
            tf.float32,
            name="decoder_past_key_values",
        ),
        "encoder_past_key_values": tf.TensorSpec(
            (
                4,
                2,
                None,
                None,
                None,
                64,
            ),  # (layers, key-values, batch_size, num_heads, encoder_sequence_length, embed_size_per_head)
            tf.float32,
            name="encoder_past_key_values",
        ),
    }
]


class WhisperExport(TFWhisperForConditionalGeneration):
    def to_logits(self, decoder_last_hidden_state):
        # Decoder and encoder embeddings are tied, convert to logits
        lm_logits = tf.matmul(
            decoder_last_hidden_state,
            self.get_output_embeddings().weights,
            transpose_b=True,
        )
        return lm_logits

    def convert_cache_tensors_to_tuple(
        self, decoder_cache_tensors, encoder_cache_tensors
    ):
        return tuple(
            [
                tuple(
                    [decoder_cache_tensors[i, j] for j in range(2)]
                    + [encoder_cache_tensors[i, j] for j in range(2)]
                )
                for i in range(self.config.decoder_layers)
            ]
        )

    @staticmethod
    def make_cache_tensors(past_key_values):
        return tf.stack(
            [[k for k in tensors[0:2]] for tensors in past_key_values]
        ), tf.stack([[k for k in tensors[2:4]] for tensors in past_key_values])

    def make_encoder_serving(self):
        @tf.function(input_signature=encoder_signature)
        def encoder_serving(inputs):
            return {"last_hidden_state": self.get_encoder()(inputs).last_hidden_state}

        return convert_variables_to_constants_v2(
            encoder_serving.get_concrete_function()
        )

    def make_decoder_init_serving(self):
        @tf.function(input_signature=decoder_init_signature)
        def decoder_init_serving(inputs):
            outputs = self.get_decoder()(
                inputs["decoder_input_ids_init"],
                encoder_hidden_states=inputs["encoder_state_init"],
            )

            lm_logits = self.to_logits(outputs[0])
            decoder_past_key_values, encoder_past_key_values = self.make_cache_tensors(
                outputs.past_key_values
            )
            return {
                "logits_init": lm_logits,
                "decoder_past_key_values": decoder_past_key_values,
                "encoder_past_key_values": encoder_past_key_values,
            }

        return convert_variables_to_constants_v2(
            decoder_init_serving.get_concrete_function()
        )

    def make_decoder_serving(self):
        @tf.function(input_signature=decoder_cache_signature)
        def decoder_serving(inputs):
            past_key_values_in = self.convert_cache_tensors_to_tuple(
                inputs["decoder_past_key_values"], inputs["encoder_past_key_values"]
            )
            outputs = self.get_decoder()(
                inputs["decoder_input_ids"],
                encoder_hidden_states=inputs["encoder_state"],
                past_key_values=past_key_values_in,
                use_cache=True,
            )

            lm_logits = self.to_logits(outputs[0])
            (decoder_past_key_values, _) = self.make_cache_tensors(
                outputs.past_key_values
            )
            return {
                "logits": lm_logits,
                "decoder_past_key_values": decoder_past_key_values,
            }

        return convert_variables_to_constants_v2(
            decoder_serving.get_concrete_function()
        )


class ModelWrapping(tf.Module):
    """
    Wrapper class for specifying signatures
    """

    @tf.function(input_signature=encoder_signature)
    def encoder_serving(self, inputs):
        return {
            "last_hidden_state": self.__encoder_serving_f(
                input_features=inputs["input_features"],
            )[0]
        }

    @tf.function(input_signature=decoder_init_signature)
    def decoder_init_serving(self, inputs):
        decoder_output = self.__decoder_init_serving_f(
            decoder_input_ids_init=inputs["decoder_input_ids_init"],
            encoder_state_init=inputs["encoder_state_init"],
        )
        return {
            "logits_init": decoder_output[2],
            "decoder_past_key_values_init": decoder_output[0],
            "encoder_past_key_values_init": decoder_output[1],
        }

    @tf.function(input_signature=decoder_cache_signature)
    def decoder_cache_serving(self, inputs):
        decoder_output = self.__decoder_cache_serving_f(
            decoder_input_ids=inputs["decoder_input_ids"],
            encoder_state=inputs["encoder_state"],
            decoder_past_key_values=inputs["decoder_past_key_values"],
            encoder_past_key_values=inputs["encoder_past_key_values"],
        )
        return {
            "logits": decoder_output[1],
            "decoder_past_key_values_out": decoder_output[0],
        }

    def export(self, export_path):
        signatures = {
            "encoder": self.encoder_serving,
            "decoder_init": self.decoder_init_serving,
            "decoder_with_past": self.decoder_cache_serving,
        }

        tf.saved_model.save(self, export_path, signatures=signatures)

    def __init__(
        self, encoder_serving_f, decoder_init_serving_f, decoder_cache_serving_f
    ):
        self.__encoder_serving_f = encoder_serving_f
        self.__decoder_init_serving_f = decoder_init_serving_f
        self.__decoder_cache_serving_f = decoder_cache_serving_f


whisper_model = WhisperExport.from_pretrained(MODEL_NAME, from_pt=True)

model_wrapping = ModelWrapping(
    whisper_model.make_encoder_serving(),
    whisper_model.make_decoder_init_serving(),
    whisper_model.make_decoder_serving(),
)

model_wrapping.export(EXPORT_PATH)
whisper_model.config.save_pretrained(assets_folder)

processor = WhisperProcessor.from_pretrained(MODEL_NAME)
processor.save_pretrained(assets_folder)

All PyTorch model weights were used when initializing WhisperExport.

All the weights of WhisperExport were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use WhisperExport for predictions without further training.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {EXPORT_PATH}

total 259576
drwxr-xr-x 2 root root      4096 Aug 22 16:34 assets
-rw-r--r-- 1 root root        55 Aug 22 16:34 fingerprint.pb
-rw-r--r-- 1 root root 265791838 Aug 22 16:34 saved_model.pb
drwxr-xr-x 2 root root      4096 Aug 22 16:34 variables


In [ ]:
!ls -l {EXPORT_PATH}/assets

total 1572
-rw-r--r-- 1 root root    2082 Aug 22 16:34 added_tokens.json
-rw-r--r-- 1 root root    2276 Aug 22 16:34 config.json
-rw-r--r-- 1 root root  493864 Aug 22 16:34 merges.txt
-rw-r--r-- 1 root root   52666 Aug 22 16:34 normalizer.json
-rw-r--r-- 1 root root     339 Aug 22 16:34 preprocessor_config.json
-rw-r--r-- 1 root root    2077 Aug 22 16:34 special_tokens_map.json
-rw-r--r-- 1 root root     805 Aug 22 16:34 tokenizer_config.json
-rw-r--r-- 1 root root 1036584 Aug 22 16:34 vocab.json


## Import and Save WhisperForCTC in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script
- Additionally, we will need to upgrade Spark to version 3.4.1

In [ ]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash
! pip install -U pyspark==3.4.1

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp

# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `WhisperForCTC` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `WhisperForCTC` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [ ]:
from sparknlp.annotator import *

whisper = (
    WhisperForCTC.loadSavedModel(f"{EXPORT_PATH}", spark)
    .setInputCols("audio_assembler")
    .setOutputCol("text")
)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
whisper.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your WhisperForCTC model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

total 259600
drwxr-xr-x 7 root root      4096 Aug 22 16:57 fields
drwxr-xr-x 2 root root      4096 Aug 22 16:56 metadata
-rw-r--r-- 1 root root 265819147 Aug 22 16:57 whisper_ctc_tensorflow


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny WhisperForCTC model 😊

In [ ]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

audioAssembler = AudioAssembler() \
    .setInputCol("audio_content") \
    .setOutputCol("audio_assembler")

speechToText = WhisperForCTC.load(f"{MODEL_NAME}_spark_nlp")

pipeline = Pipeline().setStages([audioAssembler, speechToText])

audio_path = "../../../src/test/resources/audio/txt/librispeech_asr_0.txt"

with open(audio_path) as file:
    raw_floats = [float(data) for data in file.read().strip().split("\n")]

processedAudioFloats = spark.createDataFrame([[raw_floats]]).toDF("audio_content")

result = pipeline.fit(processedAudioFloats).transform(processedAudioFloats)
result.select("text.result").show(truncate = False)

+------------------------------------------------------------------------------------------+
|result                                                                                    |
+------------------------------------------------------------------------------------------+
|[ Mr. Quilter is the apostle of the middle classes and we are glad to welcome his gospel.]|
+------------------------------------------------------------------------------------------+



That's it! You can now go wild and use hundreds of WhisperForCTC models from HuggingFace 🤗 in Spark NLP 🚀
